# EMS/CAN DATABASE BUILDER

## 사용자 설정

| SYSTEM CONSTANT | TYPE |    EXTENSION   | PRE-CONDITION | DEFAULT |   ARGUMENTS  |  FUNCTIONALITY  |
|-----------------|------|----------------|---------------|---------|--------------|-----------------|
| ENGINEERING_DB  | path | *.json, *.xlsx |       -       |   ""    |       -      | 자체 DB 전체 경로, 빈 값의 경우 SVN 공식 DB 사용 |
| SYSTEM_TYPE     | str  |        -       |       -       |  "ICE"  | "ICE", "HEV" | 시스템 사양 정의: ICE 또는 HEV |

In [ ]:
import emscan as ec

ENGINEERING_DB  = r"" 
POWERTRAIN_TYPE = "HEV"

## 초기화

In [ ]:
ec.SourceControl.update(
    ec.PATH.SVN.CAN.DB,
    ec.PATH.SVN.CAN.MD,
    ec.PATH.SVN.BUILD.SDD,
    ec.PATH.SVN.BUILD.CONF
)

if ENGINEERING_DB:
    ec.DB.reset(ENGINEERING_DB)
print("DB INFO:")
ec.DBio.baseline(ENGINEERING_DB)

## DBC

In [ ]:
# TODO
# Vector DBC Write 신규 작성

## 모델(MD)

### %ComDef / %ComDef_HEV

예외사항 1. 신규 메시지 추가의 경우<br>
1. 소스 모델 내 "_[메시지명]"의 신규 프로세스 생성 후 Export
2. Export된 모델 경로로 COMDEF_PATH 시스콘 값 업데이트 <br>
```COMDEF_PATH = D:\\ETASData\ASCET6.1\Export\ComDef\ComDef.main.amd # ICE CASE```<br>
```COMDEF_PATH = D:\\ETASData\ASCET6.1\Export\ComDef_HEV\ComDef_HEV.main.amd # HEV CASE```


In [ ]:
# ---------------------------------------------------------------------------------------------
# 기본 소스 사용 시
# ---------------------------------------------------------------------------------------------
SOURCE_COMDEF = ec.PATH.SVN.CAN.file(f"ComDef{'' if POWERTRAIN_TYPE == 'ICE' else '_HEV'}.zip")

# ---------------------------------------------------------------------------------------------
# 사용자 변경 소스 사용 시
# ---------------------------------------------------------------------------------------------
# SOURCE_COMDEF = r"D:\ETASData\ASCET6.1\Export\ComDef\ComDef.main.amd"

exclusive = ["EMS", "CVVD", "MHSG", "NOx"]
if POWERTRAIN_TYPE == "ICE":
    exclusive += ["BMS", "LDC"]

ec.DB.dev_mode(POWERTRAIN_TYPE)
ec.DB.constraint(~ec.DB.ECU.isin(exclusive))
ec.ComDef(source=SOURCE_COMDEF, database=ec.DB) \
  .write()

### %ComRx / %ComRx_HEV

예외사항 1. 신규 OS 추가의 경우
1. 소스 모델 내 OS Task 신규 생성 후 Export
2. Export된 모델 경로로 COMRX_PATH 시스콘 값 업데이트 <br>
```COMDEF_PATH = D:\\ETASData\ASCET6.1\Export\ComRx\ComRx.main.amd # ICE CASE```<br>
```COMDEF_PATH = D:\\ETASData\ASCET6.1\Export\ComRx_HEV\ComRx_HEV.main.amd # HEV CASE```

In [ ]:
# ---------------------------------------------------------------------------------------------
# 기본 소스 사용 시
# ---------------------------------------------------------------------------------------------
SOURCE_COMRX = ec.PATH.SVN.CAN.file(f"ComRx{'' if POWERTRAIN_TYPE == 'ICE' else '_HEV'}.zip")

# ---------------------------------------------------------------------------------------------
# 사용자 변경 소스 사용 시
# ---------------------------------------------------------------------------------------------
# SOURCE_COMRX = r"D:\ETASData\ASCET6.1\Export\ComRx\ComRx.main.amd"


exclusive = ["EMS", "CVVD", "MHSG", "NOx"]
if POWERTRAIN_TYPE == "ICE":
    exclusive += ["BMS", "LDC"]

ec.DB.dev_mode(POWERTRAIN_TYPE)
ec.DB.constraint(~ec.DB.ECU.isin(exclusive))
ec.ComX(source=SOURCE_COMRX, database=ec.DB) \
  .write()

## 사양서(SDD)

* 주의: C:\User\Administrator\Downloads 또는 사용자 다운로드 경로의 사양서 파일이 열려있지 않아야 합니다.

In [ ]:
ec.DB.reset(ENGINEERING_DB)
ec.DB.constraint(ec.DB[f'{POWERTRAIN_TYPE} Channel'] != "")
ec.generateSDD(ec.DB)

## 테스트케이스(TC)

In [ ]:
TESTCASE_OPTION = {
    "Requirement - Traceability"    : ec.DB.traceability,
    "Test SW"                       : "TX4T9MTN9L1N @C20", 
    "Test HW"                       : "Kappa 1.5 TGDI",
    "Test Vehicle / Engine / HIL"   : "SP3i PROTO(BS6)",
    "Test Environment"              : "Dynamic(Chamber)",
    "Remark / Comment"              : f"AUTOMATIC TEST CASE V{ec.TODAY}",
    "Measure / Log File (.dat)"     : "",
    "MDA Configuration File (.xda)" : "",
    "Experiment File (.exp)"        : "",
}

### 1. 수신 디코드(DECODE)

* 전수 생성의 경우 

In [ ]:
testcase = ec.testCaseRxDecode(ec.DB("ABS_ESC_01_10ms"), **TESTCASE_OPTION)
testcase.generate()
testcase.saveToTestReport()

### 2. 송신 인터페이스